# Übungsaufgaben 6.2: Encoding



## 1. Aufgabe `count_glyphs`
Schreiben Sie ein Programm (awk, Shell-Skript oder Python), das die
Anzahl der Buchstaben in einem Text zählt, wobei kombinierende Zeichen
nicht berücksichtigt werden. Das Programm soll von `stdin` lesen und
die Anzahl der Zeichen nach `stdout` schreiben (wie z.B. `wc`).

```bash
$ echo -n "Hallo aus München!" | uconv -f utf8 -t utf8 -x nfd | ./count_glyphs.py
18
$
```


In [ ]:
# nfd: decomposition 
# nfc: composition

### Lösung 1: count_glyphs.bash (mit uconv)

- uconv: re-composing (nfc) decomposed encoding (aus dem Input: nfd)

In [ ]:
%%bash
#!/bin/bash
uconv -f utf8 -t utf8 -x nfc | wc -m

### Lösung 2 (mit iconv)


#### alternativ mit iconv und utf8-mac Kodierung (nur MAC OS):

```bash
$ echo -n "Hallo aus München!" | iconv -f utf8 -t utf8-mac | ./count_glyphs.py
18
$
```



In [ ]:
# utf8-mac: decomposed
# utf8: composed

In [1]:
%%bash
echo -n "Mü" | iconv -f utf8 -t utf8-mac | wc -m

       3


#### Lösung:

In [32]:
%%bash
echo -n "Mü" | iconv -f utf8 -t utf8-mac | iconv -f utf8-mac -t utf8 | wc -m

       2


### Erläuterung:

#### composed (default):

In [2]:
%%bash
echo -n "ü" | iconv -f utf8 -t utf8 | hexdump -C

00000000  c3 bc                                             |..|
00000002


In [4]:
%%bash
echo -n "ü" | iconv -f utf8 -t utf8 | wc -m  #count chars

       1


In [3]:
%%bash
echo -n "ü" | iconv -f utf8 -t utf8 | wc -c  #count bytes

       2


#### decomposed:

In [5]:
%%bash
echo -n "ü" | iconv -f utf8 -t utf8-mac | hexdump -C

00000000  75 cc 88                                          |u..|
00000003


In [6]:
%%bash
#zum Vergleich (u = 75)
echo -n "u" | iconv -f utf8 -t utf8-mac | hexdump -C

00000000  75                                                |u|
00000001


In [8]:
%%bash
echo -n "ü" | iconv -f utf8 -t utf8-mac | wc -m #count chars
# M + u + combining character

       2


In [7]:
%%bash
echo -n "ü" | iconv -f utf8 -t utf8-mac | wc -c #count bytes

       3


### Lösung 3: count_glyphs.py

In [ ]:
#!/usr/bin/env python3
import unicodedata
import sys

## LOESUNG A: composing mit unicodedata (funktioniert nicht bei allen kombinierenden Zeichen)
# nfc = unicodedata.normalize('NFC', sys.stdin.read())
# print(len(nfc))


## LOESUNG B: nur nicht kombinierende Zeichen zählen
count = 0
other = 0
for chr in sys.stdin.read():
    x = unicodedata.combining(chr)
    if x == 0:
        count += 1
    else: 
        other += 1
print("Buchstaben: ", count)
print("kombinierende Zeichen: ", other)

#### Usage:

In [40]:
%%bash
echo -n "Mü" | iconv -f utf8 -t utf8-mac | ./count_glyphs.py

Buchstaben:  2
kombinierende Zeichen:  1


#### nicht alle kombinierten Unicode-Characters werden erkannt:

In [69]:
%%bash
echo -n "🇦🇩" | iconv -f utf8 -t utf8-mac | ./count_glyphs.py

Buchstaben:  2
kombinierende Zeichen:  0


In [85]:
nfc = unicodedata.normalize('NFC', "🇦🇩")
print(len(nfc))

2


In [14]:
%%bash
echo -n "🇦🇩" | iconv -f utf8 -t utf8 | hexdump -C

00000000  f0 9f 87 a6 f0 9f 87 a9                           |........|
00000008


In [11]:
"🇦🇩".encode() #A-Ländercode D-Ländercode (combining character)

b'\xf0\x9f\x87\xa6\xf0\x9f\x87\xa9'

## 2. Aufgabe `pchars`
Schreiben Sie ein Programm (awk, Shell-Skript oder Python), das für
jedes Zeichen in einem Text zeilenweise, das Zeichen, seine
Unicodenummer und seine Darstellung als UTF-8 kodiertes Zeichen
(hexadezimal) ausgibt. Das Programm soll von `stdin` lesen und die
Anzahl der Zeichen nach `stdout` schreiben.

```bash
$ echo 'Aü' | ./pchars.py
A U+0041 0x41
ü U+00FC 0xc3bc
$
```

#### pchars.py:

In [ ]:
#!/usr/bin/env python3
import sys

for c in sys.stdin.read():
    print(f"{c} U+{ord(c):04X}", end=" ") #Integer-Wert von c (ord()) in hexadezimal umwandeln (mit format-String)
    bs = c.encode(encoding='UTF-8') #umwandeln UTF-8-Kodierung in Byte-Array
    for b in bs:
        print(f"{b:02x}", end="") # Ausgabe hexadezimale UTF-8-Kodierung
    print()

#### Usage:

In [15]:
%%bash
echo -n "Aü" | ./pchars.py  #0a: newline

A U+0041 41
ü U+00FC c3bc


In [16]:
%%bash
echo -n "Aü" | iconv -f utf8 -t utf8-mac | ./pchars.py

A U+0041 41
u U+0075 75
̈ U+0308 cc88
